
<h1 style='background-color:Green; font-family:newtimeroman; font-size:180%; text-align:center; border-radius: 15px 50px;' >What is Inception v3 model? </h1>

##### Inception v3 is a widely-used image recognition model that has been shown to attain greater than 78.1% accuracy on the ImageNet dataset. The model is the culmination of many ideas developed by multiple researchers over the years.

<img src="https://www.researchgate.net/profile/Masoud-Mahdianpari/publication/326421398/figure/fig6/AS:649353890889730@1531829440919/Schematic-diagram-of-InceptionV3-model-compressed-view.png" width="800px">




<h1 style='background-color:Green; font-family:newtimeroman; font-size:180%; text-align:center; border-radius: 15px 50px;' > Aerial Cactus Identification </h1>

### Data Description
This dataset contains a large number of 32 x 32 thumbnail images containing aerial photos of a columnar cactus (Neobuxbaumia tetetzo). Kaggle has resized the images from the original dataset to make them uniform in size. The file name of an image corresponds to its id.


You must create a classifier capable of predicting whether an images contains a cactus.





<img src="https://miro.medium.com/max/1000/0*vyjWB63YjS-qZyGC" width="800px">




### Files
* train/ - the training set images
* test/ - the test set images (you must predict the labels of these)
* train.csv - the training set labels, indicates whether the image has a cactus (has_cactus = 1)
* sample_submission.csv - a sample submission file in the correct format


#### Dataset Link

##### [Here](https://www.kaggle.com/c/aerial-cactus-identification/data)

In [ ]:
# Import modules

import os
import zipfile
import random
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import cv2
from tensorflow import keras
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print(os.listdir("../input/aerial-cactus-identification/"))

In [ ]:
with zipfile.ZipFile("../input/aerial-cactus-identification/train.zip","r") as z:
    z.extractall("/kaggle/temp/")
with zipfile.ZipFile("../input/aerial-cactus-identification/test.zip","r") as z:
    z.extractall("/kaggle/temp/test/")

print(len(os.listdir("../temp/train")))
print(len(os.listdir("../temp/test/test")))

In [ ]:
train = "../temp/train"
test = "../temp/test"
labels = pd.read_csv('../input/aerial-cactus-identification/train.csv')

labels.has_cactus = labels.has_cactus.astype(str)
print(labels['has_cactus'].value_counts())

In [ ]:
# Plot random sample of training images

rand_images = random.sample(os.listdir(train), 16)

fig = plt.figure(figsize=(20,4))
for i, im in enumerate(rand_images):
    plt.subplot(2, 8, i+1)
    im = cv2.imread(os.path.join(train, im))
    plt.imshow(im)
    plt.axis('off')
plt.show()

In [ ]:
# Split training data into training and validation sets

# Could use sklearn.model_selection.train_test_split instead

validation_split = 0.8
idxs = np.random.permutation(range(len(labels))) < validation_split*len(labels)

train_labels = labels[idxs]
val_labels = labels[~idxs]
print(len(train_labels), len(val_labels))

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255, horizontal_flip=True, 
                                                             rotation_range=5,
                                                             shear_range=0.2,
                                                             height_shift_range=0.2,
                                                             width_shift_range=0.2,
                                                             vertical_flip=True,
                                                            fill_mode='nearest')

batch_size = 64

train_generator = train_datagen.flow_from_dataframe(train_labels,directory=train,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(150,150))
val_generator = train_datagen.flow_from_dataframe(val_labels,directory=train,x_col='id',
                                                    y_col='has_cactus',class_mode='binary',batch_size=batch_size,
                                                    target_size=(150,150))

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=(150,150,3),include_top=False,weights="imagenet")

In [ ]:
# Freezing Layers

for layer in base_model.layers[:-10]:
    layer.trainable=False

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-6)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=1, patience=3)

In [ ]:
model.compile(optimizer='Adam', loss = keras.losses.binary_crossentropy,metrics=METRICS)

In [ ]:
%time
history=model.fit(train_generator,validation_data=val_generator,epochs = 10,verbose = 1,callbacks=[lrd,mcp,es])

In [ ]:
# Use trained model to make predication on test data

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

test_generator = test_datagen.flow_from_directory(
    directory = test,
    target_size = (150,150),
    batch_size = 1,
    class_mode = None,
    shuffle = False)

odds = model.predict(test_generator)

In [ ]:
# Create submission file

submission = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
df = pd.DataFrame({'id': submission['id']})
df['has_cactus'] = odds
df.to_csv("submission.csv", index=False)